<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/LSTM_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación Tensorflow



In [ ]:

pip install tensorflow==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninst

# Carga de Datasets


In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
import matplotlib.pyplot as plt
import os, psutil

url1 = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset1CodificadoV2.csv'
data1 = pd.read_csv(url1)
data1 = data1.drop('Unnamed: 0',axis=1)
data1
#data1 = data1[data1['Inputs'] != 0]
data1
url2 = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset2Codificado.csv'
data2 = pd.read_csv(url2)
#data2
process = psutil.Process()
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")
data1

Memory usage: 427.94140625 megabytes


,Month,Day,Day_Week_Number,Holidays,Hour,Minute,Inputs,Outputs,Station_(02000)Cabecera Autopista Norte,Station_(02300)Calle 100,Station_(03000)Portal Suba,Station_(04000)Cabecera Calle 80,Station_(05000)Portal Américas,Station_(06000)Portal Eldorado,Station_(07000)Portal Sur,Station_(08000)Portal Tunal,Station_(09000)Cabecera Usme,Station_(10000)Portal 20 de Julio
0,8,6,7,0.0,3,45,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,8,6,7,0.0,3,45,0,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,8,6,7,0.0,3,45,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,8,6,7,0.0,3,45,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,8,6,7,0.0,3,45,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437429,8,27,7,0.0,23,45,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
437430,8,27,7,0.0,23,45,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
437431,8,27,7,0.0,23,45,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
437432,8,27,7,0.0,23,45,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


#LSTM Grid Search 80%-20%

In [ ]:
!pip install scikeras
!pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.optimizers import Adam, RMSprop
import time

start_time = time.time()
# Asume que 'data1' es tu DataFrame y que ya ha sido definido.
data1
# Preprocesamiento de datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data1)
# La columna objetivo ('Inputs') está en la posición 6, ajustamos la selección de X e y aquí
X = np.delete(scaled_data, 6, axis=1)  # Elimina la columna objetivo de las características
y = scaled_data[:, 6]  # Selecciona la columna objetivo

# Función para reestructurar los datos para el LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Definir los time_steps
time_steps = 3
X_lstm, y_lstm = create_dataset(X, y, time_steps)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

# Función para crear el modelo adaptada para SciKeras
def create_model(optimizer='adam', units=50):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=units))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[MeanSquaredError(), MeanAbsoluteError()])
    return model

# Envoltura del modelo Keras para su uso con GridSearchCV, utilizando SciKeras
model = KerasRegressor(model=create_model, optimizer='adam', units=50, verbose=0)

# Define el grid de hiperparámetros a probar
param_grid = {
    'model__optimizer': ['adam', 'rmsprop'],
    'model__units': [50, 100],
    'batch_size': [64, 128],
    'epochs': [10, 20]
}

# Crea y ejecuta el GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Muestra los resultados del GridSearch
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

# Obtener el mejor modelo entrenado desde GridSearchCV y hacer predicciones
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)

# Calcular y mostrar MSE, MAE y R^2 usando las predicciones
mse = np.mean((y_test - y_pred)**2)
mae = np.mean(np.abs(y_test - y_pred))
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R^2: {r2}')

process = psutil.Process()
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")

end_time = time.time()
print(f"Tiempo de ejecución: {end_time - start_time} segundos")

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
Mejor: 0.053144 usando {'batch_size': 64, 'epochs': 20, 'optimizer': 'rmsprop', 'units': 100}


# LSTM Grid Search 70%-30%

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.optimizers import Adam, RMSprop
import time

start_time = time.time()
# Asume que 'data1' es tu DataFrame y que ya ha sido definido.
data1
# Preprocesamiento de datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data1)
# La columna objetivo ('Inputs') está en la posición 6, ajustamos la selección de X e y aquí
X = np.delete(scaled_data, 6, axis=1)  # Elimina la columna objetivo de las características
y = scaled_data[:, 6]  # Selecciona la columna objetivo

# Función para reestructurar los datos para el LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Definir los time_steps
time_steps = 3
X_lstm, y_lstm = create_dataset(X, y, time_steps)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.3, random_state=42)

# Función para crear el modelo adaptada para SciKeras
def create_model(optimizer='adam', units=50):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=units))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[MeanSquaredError(), MeanAbsoluteError()])
    return model

# Envoltura del modelo Keras para su uso con GridSearchCV, utilizando SciKeras
model = KerasRegressor(model=create_model, optimizer='adam', units=50, verbose=0)

# Define el grid de hiperparámetros a probar
param_grid = {
    'model__optimizer': ['adam', 'rmsprop'],
    'model__units': [50, 100],
    'batch_size': [64, 128],
    'epochs': [10, 20]
}

# Crea y ejecuta el GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Muestra los resultados del GridSearch
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

# Obtener el mejor modelo entrenado desde GridSearchCV y hacer predicciones
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)

# Calcular y mostrar MSE, MAE y R^2 usando las predicciones
mse = np.mean((y_test - y_pred)**2)
mae = np.mean(np.abs(y_test - y_pred))
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R^2: {r2}')

process = psutil.Process()
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")

end_time = time.time()
print(f"Tiempo de ejecución: {end_time - start_time} segundos")

In [ ]:
Mejor: 0.052403 usando {'batch_size': 64, 'epochs': 20, 'model__optimizer': 'adam', 'model__units': 100}
